In [ ]:
import os

from astropy.time import Time, TimeDelta
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
import numpy as np
import pandas as pd
import yaml

from lsst_efd_client import EfdClient

%matplotlib inline

In [ ]:
ifilename = "cchs_heartbeat.yaml"
efd_name = "ncsa_teststand_efd"
start_time_str = "2020-06-06T00:00:00"
end_time_str = "2020-06-08T18:00:00"

In [ ]:
with open(ifilename) as ifile:
    info = yaml.safe_load(ifile)
try:
    info['expectedRate'] = eval(info['expectedRate'])
except TypeError:
    pass

In [ ]:
client = EfdClient(efd_name)

In [ ]:
time_increment = TimeDelta(600, format='sec', scale='tai')
start_time = Time(start_time_str, scale='tai')
end_time = Time(end_time_str, scale='tai') + time_increment
expected_num_messages = round(time_increment.sec) * info['expectedRate']

df = await client.select_time_series(info['tablename'], info['columnName'], start_time, end_time)

In [ ]:
df1 = df[info['columnName']].resample(time_increment.datetime).count().to_frame()

In [ ]:
df1.rename(columns={info['columnName']: "Received Messages"}, inplace=True)

In [ ]:
df1["Expected Messages"] = np.full((df1.shape[0],), expected_num_messages)
df1["Percent Loss"] = (1.0 - df1["Received Messages"] / df1["Expected Messages"]) * 100.0

In [ ]:
data_em = hv.Curve(df1, 'index', 'Expected Messages', label='Expected')
data_rm = hv.Curve(df1, 'index', 'Received Messages', label='Received')

In [ ]:
layout = data_em * data_rm
layout.opts(hv.opts.Curve(axiswise=True, height=300, width=800, padding=(0.02, 0.025)),
            hv.opts.Overlay(title=info["plotTitle"], xlabel='Date', ylabel='Messages',
                            legend_position='bottom_left'))

In [ ]:
data_pl = hv.Curve(df1, 'index', 'Percent Loss')
layout = data_pl
layout.opts(hv.opts.Curve(height=300, width=800, padding=(0.02, 0.025), title=info["plotTitle"],
                          xlabel='Date'))